In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [2]:
import requests, folium
from urllib.parse import quote
from bs4 import BeautifulSoup

In [3]:
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
url = 'https://www.ggpolice.go.kr/main/html.do?menu=HC44'
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe') # chromedriver 위치
driver.get(url)

In [4]:
soup = BeautifulSoup(driver.page_source)
lis = soup.select('li.road')
len(lis)

31

In [5]:
li = lis[0]
li

<li class="road selected">
<a href="https://www.ggpolice.go.kr/swjb/html.do?menu=HC24">수원중부서</a>
</li>

In [6]:
href = li.select_one('a')['href']
href

'https://www.ggpolice.go.kr/swjb/html.do?menu=HC24'

In [7]:
base_url = 'https://www.ggpolice.go.kr/'
sub_url  = f'{href}'

In [8]:
res = requests.get(href)
sub_soup = BeautifulSoup(res.text)

In [9]:
len(sub_soup.select('li'))

128

In [10]:
sub_lis = sub_soup.select('.basic_list7 > li')
sub_lis

[<li>기관명 : 수원중부경찰서</li>,
 <li>도로명 주소 : 경기도 수원시 장안구 정자천로 199 (우)16335</li>,
 <li>대표전화 : 경찰민원 콜센터 182</li>]

In [11]:
police = sub_lis[0].get_text().split(':')[1]
police

' 수원중부경찰서'

In [12]:
sub_lis[1]

<li>도로명 주소 : 경기도 수원시 장안구 정자천로 199 (우)16335</li>

In [13]:
addr_dict = {}
for li in sub_lis[1]:
    item = li.get_text().split(':')
    addr_dict[item[0].strip()] = item[1].split('(')[0].strip()
addr_dict

{'도로명 주소': '경기도 수원시 장안구 정자천로 199'}

In [14]:
addr = sub_lis[1].get_text().split(':')[1].split('(')[0].strip()
addr

'경기도 수원시 장안구 정자천로 199'

In [15]:
    # for li in sub_lis[1]:
    #     item = li.get_text().split(':')
    #     addr_dict[item[0].strip()] = item[1].split('(')[0].strip()

In [16]:
# url = 'https://www.ggpolice.go.kr/main/html.do?menu=HC44'
# soup = BeautifulSoup(driver.page_source)
# lines = []
# lis = soup.select('li.road')
# for li in lis:
#     driver.get(url)
#     href = li.select_one('a')['href']
    
#     res = requests.get(href)
#     sub_soup = BeautifulSoup(res.text)
#     sub_lis = sub_soup.select('.basic_list7 > li') # 
#     if len(sub_lis) == 0 : # list의 개수가없는것은 주소랑 없는 예외..수원남부경찰서
#         continue # 위로다시올라감
#     police = sub_lis[0].get_text().split(':')[1]
#     addr = sub_lis[1].get_text().split(':')[1].split('(')[0].strip()

#     lines.append({'경찰서이름':police, '주소':addr})
# driver.close()

In [17]:
url = 'https://www.ggpolice.go.kr/main/html.do?menu=HC44'
soup = BeautifulSoup(driver.page_source)
lines = []
lis = soup.select('li.road')
for li in lis:
    driver.get(url)
    href = li.select_one('a')['href']
    try:    
        res = requests.get(href)
        sub_soup = BeautifulSoup(res.text)
        sub_lis = sub_soup.select('.basic_list7 > li') # 
    # if len(sub_lis) == 0 : # list의 개수가없는것은 주소랑 없는 예외..수원남부경찰서
    #     continue # 위로다시올라감
        police = sub_lis[0].get_text().split(':')[1]
        addr = sub_lis[1].get_text().split(':')[1].split('(')[0].strip()

        lines.append({'경찰서이름':police, '주소':addr})
    except:
        continue
driver.close()

In [18]:
df = pd.DataFrame(lines)

In [20]:
df

,경찰서이름,주소
0,수원중부경찰서,경기도 수원시 장안구 정자천로 199
1,수원남부경찰서,경기 수원시 영통구 매봉로 52
2,수원서부경찰서,경기 수원시 권선구 서부로 1673
3,안양동안경찰서,경기도 안양시 동안구 동안로159번길 43
4,안양만안경찰서,경기도 안양시 만안구 냉천로 63
5,과천경찰서,경기 과천시 통영로 20
6,성남수정경찰서,경기 성남시 수정구 성남대로 1259
7,성남중원경찰서,경기 성남시 중원구 금빛로2번길 10
8,분당경찰서,경기 성남시 분당구 정자일로 165
9,부천소사경찰서,경기 부천시 부광로 214


In [21]:
df.to_csv('data/경기남부경찰서.csv')

- 카카오 api

In [22]:
with open('../04.지도시각화/data/kakaoapikey.txt') as f:
    kakao_key = f.read()

In [23]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json' # 카카오 Request get
addr = '경기도 수원시 영통구 월드컵로 199 (원천동)'
url = f'{base_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EA%B2%BD%EA%B8%B0%EB%8F%84%20%EC%88%98%EC%9B%90%EC%8B%9C%20%EC%98%81%ED%86%B5%EA%B5%AC%20%EC%9B%94%EB%93%9C%EC%BB%B5%EB%A1%9C%20199%20%28%EC%9B%90%EC%B2%9C%EB%8F%99%29'

In [24]:
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [25]:
result = requests.get(url, headers=header).json() 
result

{'documents': [{'address': {'address_name': '경기 수원시 영통구 원천동 13-4',
    'b_code': '4111710200',
    'h_code': '4111755000',
    'main_address_no': '13',
    'mountain_yn': 'N',
    'region_1depth_name': '경기',
    'region_2depth_name': '수원시 영통구',
    'region_3depth_h_name': '원천동',
    'region_3depth_name': '원천동',
    'sub_address_no': '4',
    'x': '127.044142224462',
    'y': '37.279204703544'},
   'address_name': '경기 수원시 영통구 월드컵로 199',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '경기 수원시 영통구 월드컵로 199',
    'building_name': '아주대학교어학교육원',
    'main_building_no': '199',
    'region_1depth_name': '경기',
    'region_2depth_name': '수원시 영통구',
    'region_3depth_name': '원천동',
    'road_name': '월드컵로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.044142224462',
    'y': '37.279204703544',
    'zone_no': '16502'},
   'x': '127.044142224462',
   'y': '37.279204703544'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [26]:
lat = float(result['documents'][0]['y']) # 경도 문자열로나와서 float
lng = float(result['documents'][0]['x']) # 위도
lat, lng

(37.279204703544, 127.044142224462)

In [27]:
# 반복문으로 위도,경도
lat_list, lng_list = [], []
for i in df.index:
    url = f'{base_url}?query={quote(df.주소[i])}' 
    result = requests.get(url, headers=header).json() 
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [28]:
# 위도,경도 열 만들기
df['위도'] = lat_list
df['경도'] = lng_list
df

,경찰서이름,주소,위도,경도
0,수원중부경찰서,경기도 수원시 장안구 정자천로 199,37.297932,126.996317
1,수원남부경찰서,경기 수원시 영통구 매봉로 52,37.272104,127.054492
2,수원서부경찰서,경기 수원시 권선구 서부로 1673,37.258740,126.972264
3,안양동안경찰서,경기도 안양시 동안구 동안로159번길 43,37.391209,126.948866
4,안양만안경찰서,경기도 안양시 만안구 냉천로 63,37.387334,126.925974
5,과천경찰서,경기 과천시 통영로 20,37.429358,126.989987
6,성남수정경찰서,경기 성남시 수정구 성남대로 1259,37.442287,127.126662
7,성남중원경찰서,경기 성남시 중원구 금빛로2번길 10,37.442664,127.170371
8,분당경찰서,경기 성남시 분당구 정자일로 165,37.365305,127.105450
9,부천소사경찰서,경기 부천시 부광로 214,37.468723,126.824181


In [29]:
map = folium.Map(location=[df.위도.mean(),df.경도.mean()], zoom_start=10)
for i in df.index:
    folium.Marker(
        location=[df.위도[i],df.경도[i]],
        popup=folium.Popup(df.주소[i],max_width=200),
        tooltip=df.경찰서이름[i],
        icon=folium.Icon(
        color ='blue',icon_color='white',
        icon='glyphicon glyphicon-pushpin', #glyphicon 아이콘 제공 사이트
        prefix='glyphicon'
    )
    ).add_to(map)
title_html ='<h3 align="center" style ="font-size:20px">경기남부경찰서 시각화</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map